In [1]:
import sys
import re
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections.abc import Iterable
import random
from lxml import etree
import gzip

from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
import ner
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn import metrics
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
%load_ext autoreload
%autoreload 2
from unwiki import unwiki
from ner.chunker import NamedEntityChunker

In [26]:
# The results for the search for definition (currently just Wikipedia)
with open('/media/hd1/wikipedia/wiki_definitions_improved.txt', 'r') as wiki_f:
    wiki = wiki_f.readlines()

In [38]:
%%script echo Save and create xml zipped file
root = etree.Element('root')

for line in wiki:
    title, section, defin = line.split('-#-%-')
    defin = unwiki.loads(eval(defin)).replace('\n', ' ')
    def_tree = etree.Element('definition')
    stmnt = etree.SubElement(def_tree, 'stmnt')
    stmnt.text = defin
    dfndum = etree.SubElement(def_tree, 'dfndum')
    dfndum.text = title.strip()
    root.append(def_tree)
#print(etree.tostring(root, pretty_print=True).decode('utf8')) 

with gzip.open('/media/hd1/wikipedia/wiki_definitions_improved.xml.gz', 'w') as wiki_fobj:
    wiki_fobj.write(etree.tostring(root, pretty_print=True))

Save and create xml zipped file


In [5]:
def split_fields(elem):
    title = elem.find('.//dfndum').text 
    section = elem.get('name')
    defin = elem.find('.//stmnt').text
    return (title, section, defin)

plmath = []
with gzip.open('/media/hd1/planetmath/datasets/planetmath_definitions.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('article'):
        plmath.append(split_fields(art))
stacks = []
with gzip.open('/media/hd1/stacks-project/datasets/stacks-definitions.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('article'):
        try:
            stacks.append(split_fields(art))
        except AttributeError:
            print('The name of the problematic article is: {}'.format(art.attrib['name']))

# Print results
print('Found {} in the stacks dataset.'.format(len(stacks)))

The name of the problematic article is: examples.xml
The name of the problematic article is: coding.xml
The name of the problematic article is: spaces-pushouts.xml
The name of the problematic article is: guide.xml
The name of the problematic article is: moduli.xml
The name of the problematic article is: more-groupoids.xml
The name of the problematic article is: chapters.xml
The name of the problematic article is: sets.xml
The name of the problematic article is: obsolete.xml
The name of the problematic article is: examples-defos.xml
The name of the problematic article is: spaces-more-cohomology.xml
The name of the problematic article is: bibliography.xml
The name of the problematic article is: fdl.xml
The name of the problematic article is: limits.xml
The name of the problematic article is: conventions.xml
The name of the problematic article is: introduction.xml
The name of the problematic article is: quot.xml
The name of the problematic article is: desirables.xml
Found 95 in the stacks

In [16]:
random.sample(stacks, 1)

[('presheaf ',
  'sheaves.xml',
  ' Let _inline_math_ be a topological space. _item_ A presheaf _inline_math_ of sets on _inline_math_ is a rule which assigns to each open _inline_math_ a set _inline_math_ and to each inclusion _inline_math_ a map _inline_math_ such that _inline_math_ and whenever _inline_math_ we have _inline_math_. _item_ A morphism _inline_math_ of presheaves of sets on _inline_math_ is a rule which assigns to each open _inline_math_ a map of sets _inline_math_ compatible with restriction maps, i.e., whenever _inline_math_ are open the diagram commutes. _item_ The category of presheaves of sets on _inline_math_ will be denoted _inline_math_. ')]

In [6]:
# Get data and train the Sentence tokenizer
# Uses a standard algorithm (Kiss-Strunk) for unsupervised sentence boundary detection
text = ''
for i in range(550):
    #text += unwiki.loads(eval(wiki[i].split('-#-%-')[2]))
    text += plmath[i][2]
for i in range(50):
    text += stacks[i][2]
for i in range(550):
    text += unwiki.loads(eval(wiki[i].split('-#-%-')[2]))

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())
print(tokenizer._params.abbrev_types)

{'hk', 'pp', 'i.e', 'aab', 'u.n', 'scand', 'e.g', '15k', 'p.h.d', 'r.a', 'c.c.c', 'j.a', 'eqs', 'x+2', 'u.s', 'j.w', '62f', 'og', 'pl', 'inc', 'axe', 'etc', 'z-1', 's.t', '2π', 'dr', 'hyp', 'forever—e.g', 'missions', 'ton', 'vibrations', 'eq', 's^2', 'cf', 'ginebra', 'az', 'ex', 'jie', 'resp', 'a.e', 'jr', 'al', 'sow', 'brethren', 'fig', 'spacewalks', 'ca', "'is", 'vol', 'mixture', 'wings', 'n.b', 'eng', 'a.k.a', 'juniper', 'sgs', 'mr', 'i.o'}


In [7]:
# The cleaning up of the wiki markup so that it looks like normal written english
title, section, defin = wiki[850].split('-#-%-')
dclean = unwiki.loads(eval(defin))
print(title)
print(dclean)
defin

Indicator function  
The indicator function of a subset A of a set X is a function

_display_math_

defined as

_display_math_

The Iverson bracket allows the equivalent notation, _inline_math_, to be used instead of _inline_math_.

The function _inline_math_ is sometimes denoted _inline_math_, _inline_math_, _inline_math_ or even just _inline_math_. (The Greek letter _inline_math_ appears because it is the initial letter of the Greek word χαρακτήρ, which is the ultimate origin of the word characteristic.)

The set of all indicator functions on _inline_math_ can be identified with _inline_math_, the power set of _inline_math_.  Consequently, both sets are sometimes denoted by _inline_math_. This is a special case (_inline_math_) of the notation _inline_math_ for the set of all functions _inline_math_.


'  "The indicator function of a subset \'\'A\'\' of a set \'\'X\'\' is a function\\n\\n:<math>\\\\mathbf{1}_A \\\\colon X \\\\to \\\\{ 0,1 \\\\} </math>\\n\\ndefined as\\n\\n:<math>\\\\mathbf{1}_A(x) :=\\n\\\\begin{cases}\\n1 &\\\\text{if } x \\\\in A, \\\\\\\\\\n0 &\\\\text{if } x \\\\notin A.\\n\\\\end{cases}\\n</math>\\n\\nThe [[Iverson bracket]] allows the equivalent notation, <math>[x\\\\in A]</math>, to be used instead of <math>\\\\mathbf{1}_A(x)</math>.\\n\\nThe function <math>\\\\mathbf{1}_A</math> is sometimes denoted <math>I_A</math>, <math>\\\\chi_A</math>, \'\'K<sub>A</sub>\'\' or even just <math>A</math>. (The [[Greek alphabet|Greek letter]] <math>\\\\chi</math> appears because it is the initial letter of the Greek word χαρακτήρ, which is the ultimate origin of the word \'\'characteristic\'\'.)\\n\\nThe set of all indicator functions on <math>X</math> can be identified with <math>\\\\mathcal{P}(X)</math>, the [[power set]] of <math>X</math>.  Consequently, both sets are so

In [8]:
# Get the data and POS and NER tags for each definition (LONG TIME)
def put_pos_ner_tags(defl):
    def_lst = []
    for i in range(len(defl)):
        try:
            #title, section, defin_raw = wiki[i].split('-#-%-')
            #defin_all = unwiki.loads(eval(defin_raw))
            title, section, defin_all = defl[i]
            for d in tokenizer.tokenize(defin_all):
                if title.lower().strip() in d.lower():
                    pos_tokens = pos_tag(word_tokenize(d))
                    def_ner = ner.bio_tag.bio_tagger(title.strip().split(), pos_tokens)
                    other_ner = [((d[0],d[1]),d[2]) for d in def_ner]
                    tmp_dict = {'title': title,
                               'section': section,
                               'defin': d,
                               'ner': other_ner}
                    def_lst.append(tmp_dict)
        except ValueError:
            print('parsing error')
    return def_lst
def_lst = put_pos_ner_tags(stacks) + put_pos_ner_tags(plmath)

for i in range(len(wiki)):
    try:
        title, section, defin_raw = wiki[i].split('-#-%-')
        defin_all = unwiki.loads(eval(defin_raw))
        for d in tokenizer.tokenize(defin_all):
            if title.lower().strip() in d.lower():
                pos_tokens = pos_tag(word_tokenize(d))
                def_ner = ner.bio_tag.bio_tagger(title.strip().split(), pos_tokens)
                other_ner = [((d[0],d[1]),d[2]) for d in def_ner]
                tmp_dict = {'title': title,
                           'section': section,
                           'defin': d,
                           'ner': other_ner}
                def_lst.append(tmp_dict)
    except ValueError:
        print('parsing error')

In [9]:
random.sample(def_lst,1)

[{'title': 'Seifert fiber space  ',
  'section': '  Definition  ',
  'defin': 'A compact Seifert fiber space has only a finite number of exceptional fibers.',
  'ner': [(('A', 'DT'), 'O'),
   (('compact', 'JJ'), 'O'),
   (('Seifert', 'NNP'), 'B-DFNDUM'),
   (('fiber', 'NN'), 'I-DFNDUM'),
   (('space', 'NN'), 'I-DFNDUM'),
   (('has', 'VBZ'), 'O'),
   (('only', 'RB'), 'O'),
   (('a', 'DT'), 'O'),
   (('finite', 'JJ'), 'O'),
   (('number', 'NN'), 'O'),
   (('of', 'IN'), 'O'),
   (('exceptional', 'JJ'), 'O'),
   (('fibers', 'NNS'), 'O'),
   (('.', '.'), 'O')]}]

In [10]:
random.shuffle(def_lst)
training_samples = [d['ner'] for d in def_lst[:int(len(def_lst) * 0.8)]]
test_samples = [d['ner'] for d in def_lst[int(len(def_lst) * 0.8):]]
 
print("#training samples = %s" % len(training_samples) )   # training samples = 55809
print("#test samples = %s" % len(test_samples))            # test samples = 6201

#training samples = 21148
#test samples = 5288


In [11]:
#train the NER Chunking Classifier (TAKES A LONG TIME)
%time chunker = NamedEntityChunker(training_samples)

CPU times: user 23 s, sys: 208 ms, total: 23.2 s
Wall time: 23.2 s


In [12]:
# Evaluate the most common metrics on the test dataset
unpack = lambda l: [(tok, pos, ner) for ((tok, pos), ner) in l]
Tree_lst = [conlltags2tree(unpack(t)) for t in test_samples]
print(chunker.evaluate(Tree_lst))

ChunkParse score:
    IOB Accuracy:  88.4%%
    Precision:     25.6%%
    Recall:        63.5%%
    F-Measure:     36.5%%


### Other Scores

Training with an amount of the dataset and evaluating with the rest
* With 80% split of the stacks data:
```
ChunkParse score:
    IOB Accuracy:  86.5%%
    Precision:     29.0%%
    Recall:        60.0%%
    F-Measure:     39.1%%
```

* With 80% of Wiki + Planetmath data:
```
ChunkParse score:
    IOB Accuracy:  88.4%%
    Precision:     26.5%%
    Recall:        66.3%%
    F-Measure:     37.9%%
```

* With 90% of Wiki + Planetmath:
```
ChunkParse score:
    IOB Accuracy:  88.7%%
    Precision:     27.1%%
    Recall:        67.7%%
    F-Measure:     38.7%%
```

* With 90% of the dataset Planetmath data alone:
```
ChunkParse score:
    IOB Accuracy:  86.0%%
    Precision:     24.0%%
    Recall:        67.3%%
    F-Measure:     35.3%%
```
    
* With 80% of the dataset

* 60% of the data

```
ChunkParse score:
    IOB Accuracy:  91.0%%
    Precision:     30.7%%
    Recall:        63.9%%
    F-Measure:     41.5%%
```
    
```
ChunkParse score:
    IOB Accuracy:  90.6%%
    Precision:     32.4%%
    Recall:        68.7%%
    F-Measure:     44.0%%
```

* 90% of the data

```
ChunkParse score:
    IOB Accuracy:  91.2%%
    Precision:     32.0%%
    Recall:        68.0%%
    F-Measure:     43.5%%
```
    


In [13]:
# An example of a user fed definition
print(chunker.parse(pos_tag(word_tokenize("We define a Banach space as a complete vector space."))))

(S
  We/PRP
  define/VBP
  a/DT
  (DFNDUM Banach/NNP space/NN)
  as/IN
  a/DT
  (DFNDUM complete/JJ vector/NN space/NN)
  ./.)


In [14]:
def prepare_for_metrics(int_range, chunker_fn, data_set = test_samples, print_output=False):
    '''
    `int_range` is an integer range
    NEEDS A TEST_SAMPLES VARIABLE CREATED WHEN SPLITTING THE 
    TRAINING AND TESTING DATA
    Returns two vectors ready to be used in the 
    metrics classification function
    '''
    if isinstance(int_range, int):
        int_range = [int_range]
    y_true = []
    y_pred = []
    for i in int_range:
        sample = data_set[i]
        sm = [s[0] for s in sample]
        y_true_tmp = [s[1] for s in sample]
        predicted = [v[2] for v in tree2conlltags(chunker_fn.parse(sm))]
        y_true += y_true_tmp
        y_pred += predicted
        if print_output:
            for k,s in enumerate(sm):
                print('{:15} {:>10}  {:>10}'.format(s[0], y_true_tmp[k], predicted[k]))
    return y_true, y_pred

In [15]:
OO = prepare_for_metrics(19, chunker, data_set=test_samples, print_output=True)

Some                     O           O
sources                  O           O
define                   O           O
the                      O           O
exponential       B-DFNDUM    B-DFNDUM
time              I-DFNDUM    I-DFNDUM
hypothesis        I-DFNDUM    I-DFNDUM
to                       O           O
be                       O           O
the                      O           O
slightly                 O           O
weaker                   O           O
statement                O           O
that                     O           O
3-SAT                    O    B-DFNDUM
can                      O           O
not                      O           O
be                       O           O
solved                   O           O
in                       O           O
time                     O           O
_inline_math_            O           O
.                        O           O


In [16]:
y_true, predicted = prepare_for_metrics(range(len(test_samples)), chunker)
print(metrics.classification_report(y_true, predicted))

              precision    recall  f1-score   support

    B-DFNDUM       0.30      0.74      0.42      5033
    I-DFNDUM       0.27      0.82      0.40      3798
           O       0.99      0.89      0.94    153037

    accuracy                           0.88    161868
   macro avg       0.52      0.82      0.59    161868
weighted avg       0.95      0.88      0.91    161868

